In [1]:
class MealPlanner:
    def __init__(self, name: str, desc: str, difficulty: str, tags: str):
        try:
            self.name = name
            self.desc = desc
            self.diff = difficulty
            self.tags = tags
        except:
            print(name)
            raise ValueError

In [2]:
## Actions
## examples copied from hello fresh website
actions = []
actions.append(MealPlanner(name="One-Pan Beef Enchiladas Verdes with Mexican Cheese Blend & Hot Sauce Crema", difficulty="Easy", tags="Spicy, Easy Cleanup, Easy Prep", desc="When it comes to Mexican-style cuisine, burritos typically get all the glory. In our humble opinion, enchiladas are an unsung dinner hero. They’re technically easier-to-assemble burritos that get smothered in a delicious sauce, but they’re really so much more than that! Ours start with spiced beef and charred green pepper that get rolled up in warm tortillas. This winning combo gets topped with tangy salsa verde and cheese, then baked until bubbly and melty. Hear that? That’s the sound of the dinner bell!"))
actions.append(MealPlanner(name="Chicken & Mushroom Flatbreads with Gouda Cream Sauce & Parmesan", difficulty="Easy", tags="", desc="Yes we love our simple cheese pizza with red sauce but tonight, move over, marinara—there’s a new sauce in town. In this recipe, crispy flatbreads are slathered with a rich, creamy gouda-mustard sauce we just can’t get enough of. We top that off with a pile of caramelized onion and earthy cremini mushrooms. Shower with Parmesan, and that’s it. Simple, satisfying, and all in 30 minutes–a dinner idea you can’t pass up!"))
actions.append(MealPlanner(name="Sweet Potato & Pepper Quesadillas with Southwest Crema & Tomato Salsa", difficulty="Easy", tags="Veggie", desc="This quesadilla is jam-packed with flavorful roasted sweet potato and green pepper, plus two types of gooey, melty cheese (how could we choose just one?!). Of course, we’d never forget the toppings—there’s a fresh tomato salsa and dollops of spiced lime crema. Now for the fun part: piling on a little bit of everything to construct the perfect bite!"))
actions.append(MealPlanner(name="One-Pan Trattoria Tortelloni Bake with a Crispy Parmesan Panko Topping", difficulty="Easy", tags="Veggie, Easy Cleanup, Easy Prep", desc="Think a cheesy stuffed pasta can’t get any better? What about baking it in a creamy sauce with a crispy topping? In this recipe, we toss cheese-stuffed tortelloni in an herby tomato cream sauce, then top with Parmesan and panko breadcrumbs. Once broiled, it turns into a showstopping topping that’ll earn you plenty of oohs and aahs from your lucky fellow diners."))

meals = []
for action in actions:
    action_str = "title=" + action.name
    action_str = action_str.replace(":", "")
    action_str = action_str.replace("|", "")
    meals.append(action_str)


In [3]:
from langchain.chat_models import AzureChatOpenAI
import langchain
langchain.debug = False
# assuming LLM api keys have been set in the environment
# can use whatever LLM you want here doesn't have to be AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    request_timeout=10,
    max_retries=1,
    client=None,
)

llm.predict('Are you ready?')

'As an AI language model, I am always ready to assist you. How may I help you today?'

##### default chain default reward (the LLM is used to judge and rank the response)

In [21]:
import rl_chain

from langchain.prompts.prompt import PromptTemplate

_PROMPT_TEMPLATE = """Here is the description of a meal: {meal}.

You have to embed this into the given text where it makes sense. Here is the given text: {text_to_personalize}.

"""

PROMPT = PromptTemplate(
    input_variables=["meal", "text_to_personalize"], template=_PROMPT_TEMPLATE
)

chain = rl_chain.PickBest.from_llm(
    llm=llm,
    prompt=PROMPT,
    #TODO rename selectionScorer? responseScorer
    response_validator = rl_chain.PickBestAutoResponseValidator(llm=llm),
)


2023-08-16 13:36:30,738 - rl_chain.rl_chain_base - INFO - next model checkpoint = 1
2023-08-16 13:36:30,739 - rl_chain.rl_chain_base - INFO - vw command: ['--cb_explore_adf', '--quiet', '--interactions=::', '--coin', '--epsilon=0.2']
2023-08-16 13:36:30,739 - rl_chain.rl_chain_base - INFO - vw command: ['--cb_explore_adf', '--quiet', '--interactions=::', '--coin', '--epsilon=0.2']


In [22]:
response = chain.run(
    meal = rl_chain.ToSelectFrom(meals),
    User = rl_chain.BasedOn("Tom Hanks"),
    preference = rl_chain.BasedOn("Vegetarian, regular dairy is ok"),
    text_to_personalize = "This is the weeks specialty dish, our master chefs believe you will love it!",
)

print(response["response"])
rr = response["decision_metadata"]
print(f"cost: {rr.label.cost}, action: {rr.label.chosen_action}, probability: {rr.label.chosen_action_probability}, ")

shared |User Tom_Hanks 0.038023032 1.0754541 1.5410469 -0.10873903 0.45360225 0.30373034 0.12009281 1.0932133 -0.46527904 -0.1948713 -1.2035347 0.6840404 0.13216053 0.5546025 -0.008241987 0.24885857 -0.84832346 -0.13249923 0.45582128 -0.16264185 -0.71124613 -0.61281693 -0.7728447 -1.1767344 0.17085144 0.46633905 0.072064154 -1.2829635 -0.7102612 -0.14236039 -0.3591779 0.29256076 0.6572387 -0.5644682 -0.50973034 0.40182233 -0.8294649 0.031122193 -0.23652014 0.5462705 0.89050084 0.07375746 0.6905364 0.02984946 -0.21246235 -0.55669284 1.0833318 0.0004115194 -0.5103178 -1.6800869 -0.57735455 0.07232263 -1.1579039 0.95835894 -0.4887978 -0.80600464 1.4456184 -1.1326172 0.6066771 0.6585185 0.22575948 -0.64706314 0.69594693 0.77243394 -0.9289581 -0.35557735 -0.13511606 0.049885795 -0.49158183 0.5060285 1.2126106 0.007847434 -0.45823354 0.2687447 -0.34012213 -0.3844155 0.6516227 0.20087449 -0.044614024 0.06633304 0.5787183 -0.07846432 1.1852514 0.5254644 -0.10693288 0.49871612 0.12513861 0.3365

In [6]:
from langchain.prompts.prompt import PromptTemplate

_OTHER_PROMPT_TEMPLATE = """You can use the actions that were chosen by VW like so: {action}.

And use whatever other vars you want to pass into the chain at run: {some_text}. And {some_other_text}

"""


OTHER_PROMPT = PromptTemplate(
    input_variables=["action", "some_text", "some_other_text"],
    template=_OTHER_PROMPT_TEMPLATE,
)

In [7]:
import rl_chain.pick_best_chain

chain = rl_chain.PickBest.from_llm(
    llm=llm,
    model_save_dir="./models",  # where to save the model checkpoints
    prompt=OTHER_PROMPT,
    response_validator = rl_chain.PickBestAutoResponseValidator(llm=llm)
)

2023-08-16 13:08:59,211 - rl_chain.rl_chain_base - INFO - next model checkpoint = 1
2023-08-16 13:08:59,212 - rl_chain.rl_chain_base - INFO - vw command: ['--cb_explore_adf', '--quiet', '--interactions=::', '--coin', '--epsilon=0.2']
2023-08-16 13:08:59,212 - rl_chain.rl_chain_base - INFO - vw command: ['--cb_explore_adf', '--quiet', '--interactions=::', '--coin', '--epsilon=0.2']


In [8]:
response = chain.run(
    some_text = "This is some text",
    some_other_text = "This is some other text",
    action=rl_chain.ToSelectFrom(["an action", "another action", "a third action"]),
    User = rl_chain.BasedOn("Tom"),
    preference = rl_chain.BasedOn("Vegetarian")
)

print(response["response"])
rr = response["decision_metadata"]
print(f"cost: {rr.label.cost}, action: {rr.label.chosen_action}, probability: {rr.label.chosen_action_probability}, ")

shared |User Tom 0.18684192 0.096681416 1.9330019 -0.045624178 -0.30213085 0.6764795 -0.11434453 0.6398013 -0.55594736 -0.007790429 -1.5897304 0.45726803 0.21923001 0.37859926 0.4172226 0.4484079 -0.5443735 -0.2300117 0.694442 -0.2744365 -0.5103659 0.5267659 -0.4604427 -1.0941986 0.093489 -0.112085976 0.5635993 -1.4790195 -0.86593074 -0.23443902 -0.1972993 -0.34710753 0.685216 -0.33489776 -0.1780288 0.41599658 -0.837596 -0.03130729 0.08583462 -0.15782441 1.4838234 0.122970134 0.4675459 -0.022492565 -0.24572937 0.016676946 1.1944858 -0.6883199 -0.3101823 -1.4694176 -0.38760924 -0.32650197 -0.39257446 0.38152853 -0.51852304 -0.8723394 0.79032344 -0.8067972 0.41836223 0.7523418 1.2355698 -0.5021966 0.49981424 1.1434466 -0.6156824 0.17191775 0.5982433 0.0762306 -0.5276518 0.55546826 1.5133482 0.31636953 -0.5955755 0.1924602 -0.7139282 -0.7930203 0.615778 0.45224833 -0.24444725 0.3069159 0.76157904 0.3895721 1.1207767 0.33168867 -0.5153346 0.5693411 0.63743776 0.58108205 -1.268203 0.3093455

#### actions and context with multiple namespaces

In [9]:
# each action is a dictionary of namespace to action string
# this example here shows that while embedding is recommended for all features, it is not required and can be customized
action_strs_w_ns = [{"A":"an action feature", "B" : rl_chain.Embed("antoher action feature")}, {"B": "another action"}, {"C":"a third action"}]

inputs = {
    "some_text": "This is some text",
    "some_other_text": "This is some other text",
    "action" : rl_chain.ToSelectFrom(action_strs_w_ns)
}

inputs["User"] = rl_chain.BasedOn("Tom")
inputs["preference"] = rl_chain.BasedOn(rl_chain.Embed("Vegetarian"))
response = chain.run(inputs)
print(response["response"])
rr = response["decision_metadata"]
print(f"cost: {rr.label.cost}, action: {rr.label.chosen_action}, probability: {rr.label.chosen_action_probability}, ")


shared |User Tom 0.18684192 0.096681416 1.9330019 -0.045624178 -0.30213085 0.6764795 -0.11434453 0.6398013 -0.55594736 -0.007790429 -1.5897304 0.45726803 0.21923001 0.37859926 0.4172226 0.4484079 -0.5443735 -0.2300117 0.694442 -0.2744365 -0.5103659 0.5267659 -0.4604427 -1.0941986 0.093489 -0.112085976 0.5635993 -1.4790195 -0.86593074 -0.23443902 -0.1972993 -0.34710753 0.685216 -0.33489776 -0.1780288 0.41599658 -0.837596 -0.03130729 0.08583462 -0.15782441 1.4838234 0.122970134 0.4675459 -0.022492565 -0.24572937 0.016676946 1.1944858 -0.6883199 -0.3101823 -1.4694176 -0.38760924 -0.32650197 -0.39257446 0.38152853 -0.51852304 -0.8723394 0.79032344 -0.8067972 0.41836223 0.7523418 1.2355698 -0.5021966 0.49981424 1.1434466 -0.6156824 0.17191775 0.5982433 0.0762306 -0.5276518 0.55546826 1.5133482 0.31636953 -0.5955755 0.1924602 -0.7139282 -0.7930203 0.615778 0.45224833 -0.24444725 0.3069159 0.76157904 0.3895721 1.1207767 0.33168867 -0.5153346 0.5693411 0.63743776 0.58108205 -1.268203 0.3093455

##### chain with default prompt and custom reward prompt (the LLM is used to judge and rank the response)

In [10]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

human_template = "Given {preference} rank how good or bad this selection is {action}"
human_message_prompt = HumanMessagePromptTemplate.from_template(
    human_template
)

REWARD_PROMPT = ChatPromptTemplate.from_messages(
    [rl_chain.pick_best_chain.PickBestAutoResponseValidator.default_system_prompt, human_message_prompt]
)

In [11]:
import rl_chain

chain = rl_chain.PickBest.from_llm(
    llm=llm,
    prompt=OTHER_PROMPT,
    model_save_dir="./models",  # where to save the model checkpoints
    response_validator=rl_chain.PickBestAutoResponseValidator(llm=llm, prompt=REWARD_PROMPT),
)

2023-08-16 13:09:12,363 - rl_chain.rl_chain_base - INFO - next model checkpoint = 1
2023-08-16 13:09:12,364 - rl_chain.rl_chain_base - INFO - vw command: ['--cb_explore_adf', '--quiet', '--interactions=::', '--coin', '--epsilon=0.2']
2023-08-16 13:09:12,364 - rl_chain.rl_chain_base - INFO - vw command: ['--cb_explore_adf', '--quiet', '--interactions=::', '--coin', '--epsilon=0.2']


In [12]:
actions = ["an action", "another action", "a third action"]

response = chain.run(
    some_text = "Some text",
    some_other_text = "Some other text",
    action=rl_chain.ToSelectFrom(actions),
    User = rl_chain.BasedOn("Tom"),
    preference = rl_chain.BasedOn("Vegetarian"),
)
print(response["response"])
rr = response["decision_metadata"]
print(f"cost: {rr.label.cost}, action: {rr.label.chosen_action}, probability: {rr.label.chosen_action_probability}, ")

shared |User Tom 0.18684192 0.096681416 1.9330019 -0.045624178 -0.30213085 0.6764795 -0.11434453 0.6398013 -0.55594736 -0.007790429 -1.5897304 0.45726803 0.21923001 0.37859926 0.4172226 0.4484079 -0.5443735 -0.2300117 0.694442 -0.2744365 -0.5103659 0.5267659 -0.4604427 -1.0941986 0.093489 -0.112085976 0.5635993 -1.4790195 -0.86593074 -0.23443902 -0.1972993 -0.34710753 0.685216 -0.33489776 -0.1780288 0.41599658 -0.837596 -0.03130729 0.08583462 -0.15782441 1.4838234 0.122970134 0.4675459 -0.022492565 -0.24572937 0.016676946 1.1944858 -0.6883199 -0.3101823 -1.4694176 -0.38760924 -0.32650197 -0.39257446 0.38152853 -0.51852304 -0.8723394 0.79032344 -0.8067972 0.41836223 0.7523418 1.2355698 -0.5021966 0.49981424 1.1434466 -0.6156824 0.17191775 0.5982433 0.0762306 -0.5276518 0.55546826 1.5133482 0.31636953 -0.5955755 0.1924602 -0.7139282 -0.7930203 0.615778 0.45224833 -0.24444725 0.3069159 0.76157904 0.3895721 1.1207767 0.33168867 -0.5153346 0.5693411 0.63743776 0.58108205 -1.268203 0.3093455

##### other reward options

custom reward class

In [13]:
# custom reward class/function is just defining another class that inherits from RewardChecker and implementing the grade_response method
import rl_chain

class CustomResponseValidator(rl_chain.ResponseValidator):
    #grade or score the response
    def grade_response(
        self, inputs, llm_response: str
    ) -> float:
        # do whatever you want here, use whatever inputs you supplied and return reward
        reward = 1.0
        return reward
    
# set this in the chain during construction (response_validator=CustomResponseValidator()) and it will be auto-called

Asynchronus user defined reward

In [14]:
import rl_chain

chain = rl_chain.PickBest.from_llm(
    llm=llm,
    prompt=PROMPT)

# whenever you have the reward for the call, send it back to the chain to learn from

response = chain.run(text_to_personalize = "This is the weeks specialty dish, our master chefs believe you will love it!",
                     meal = rl_chain.ToSelectFrom(meals),
                     User = rl_chain.BasedOn(rl_chain.Embed("Tom")),
                     preference = rl_chain.BasedOn("Vegetarian")
                    )
print(response["response"])
rr = response["decision_metadata"]
# cost should be None here because we turned auto-checkin off
print(f"cost: {rr.label.cost}, action: {rr.label.chosen_action}, probability: {rr.label.chosen_action_probability}, ")

# learn delayed score/grade
chain.learn_delayed_reward(reward=1.0, event=rr)

print(f"cost: {rr.label.cost}")

2023-08-16 13:09:21,326 - rl_chain.rl_chain_base - WARNING - No response validator provided, which means that no reinforcement learning will be done in the RL chain unless learn_delayed_reward is called.
2023-08-16 13:09:21,328 - rl_chain.rl_chain_base - INFO - next model checkpoint = 1
2023-08-16 13:09:21,329 - rl_chain.rl_chain_base - INFO - vw command: ['--cb_explore_adf', '--quiet', '--interactions=::', '--coin', '--epsilon=0.2']


shared |User 0.18684192 0.096681416 1.9330019 -0.045624178 -0.30213085 0.6764795 -0.11434453 0.6398013 -0.55594736 -0.007790429 -1.5897304 0.45726803 0.21923001 0.37859926 0.4172226 0.4484079 -0.5443735 -0.2300117 0.694442 -0.2744365 -0.5103659 0.5267659 -0.4604427 -1.0941986 0.093489 -0.112085976 0.5635993 -1.4790195 -0.86593074 -0.23443902 -0.1972993 -0.34710753 0.685216 -0.33489776 -0.1780288 0.41599658 -0.837596 -0.03130729 0.08583462 -0.15782441 1.4838234 0.122970134 0.4675459 -0.022492565 -0.24572937 0.016676946 1.1944858 -0.6883199 -0.3101823 -1.4694176 -0.38760924 -0.32650197 -0.39257446 0.38152853 -0.51852304 -0.8723394 0.79032344 -0.8067972 0.41836223 0.7523418 1.2355698 -0.5021966 0.49981424 1.1434466 -0.6156824 0.17191775 0.5982433 0.0762306 -0.5276518 0.55546826 1.5133482 0.31636953 -0.5955755 0.1924602 -0.7139282 -0.7930203 0.615778 0.45224833 -0.24444725 0.3069159 0.76157904 0.3895721 1.1207767 0.33168867 -0.5153346 0.5693411 0.63743776 0.58108205 -1.268203 0.3093455 -0.